# Instalamos e importamos librerías

In [ ]:
!pip install pyarrow
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Ejercicio 6

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
downloaded = drive.CreateFile({'id':"1onzu1sUgeY5EwmVQnjn5nmCn6YPf6S8v"})
downloaded.GetContentFile('pagelinks.csv')

In [ ]:
downloaded = drive.CreateFile({'id':"1oS2m872fr6796N_DgVyQ1hUvGBqJb2nR"})
downloaded.GetContentFile('contents.csv')

Uso samples del 20% para ambos archivos para que alcance la RAM

In [ ]:
pagelinks_pandas = pd.read_csv('pagelinks.csv')
pagelinks_sample = pagelinks_pandas.sample(frac = 0.2)
pagelinks_sample.to_csv('pagelinks_sample.csv')

In [ ]:
contents_pandas = pd.read_csv('contents.csv')
contents_sample = contents_pandas.sample(frac = 0.2)
contents_sample.to_csv('contents_sample.csv')

In [ ]:
df = sqlContext.read.csv('pagelinks_sample.csv', header=True, inferSchema=True)
pagelinks_rdd = df.rdd

In [ ]:
pagelinks_rdd.take(5)

[Row(_c0=20759825, pl_from=594202, pl_namespace=0, pl_title='Ã\x8dberos', pl_from_namespace='0'),
 Row(_c0=14095698, pl_from=2344670, pl_namespace=0, pl_title='Library_of_Congress_Control_Number', pl_from_namespace='0'),
 Row(_c0=52800785, pl_from=5572873, pl_namespace=0, pl_title='Wikispecies', pl_from_namespace='0'),
 Row(_c0=74346250, pl_from=133341, pl_namespace=0, pl_title='Elecciones_municipales_de_EspaÃ±a_de_2015', pl_from_namespace='0'),
 Row(_c0=46168699, pl_from=80192, pl_namespace=0, pl_title='DerivaciÃ³n_(matemÃ¡tica)', pl_from_namespace='0')]

In [ ]:
df = sqlContext.read.csv('contents_sample.csv', header=True, inferSchema=True)
contents_rdd = df.rdd

In [ ]:
contents_rdd.take(5)

[Row(_c0='3286506', title='Categoría:Obras literarias en húngaro', id='7617609', namespace='14', revision_id='94345015', parent_revision=None, revision_timestamp='2016-10-16T15:12:06Z', revisor_username='Juan Mayordomo', revisor_id='591129.0', revisor_ip=None, revisor_comment='Página creada con «[[Categoría:Literatura en húngaro]] [[Categoría:Obras literarias por idioma|Hungaro]]»'),
 Row(_c0='931659', title='Tatiana Poutchek', id='2254976', namespace='0', revision_id='129453207', parent_revision='34781656.0', revision_timestamp='2020-09-21T20:04:44Z', revisor_username='EmausBot', revisor_id='1396057.0', revisor_ip=None, revisor_comment='Bot: corrección de redirección doble a [[Tatiana Púchek]]'),
 Row(_c0='3980396', title='Mendel Heymann', id='9634572', namespace='0', revision_id='130603721', parent_revision='130595868.0', revision_timestamp='2020-11-03T13:51:47Z', revisor_username='Aosbot', revisor_id='5045504.0', revisor_ip=None, revisor_comment='Mantenimiento de Control de autorida

In [ ]:
pagelinks_by_title = pagelinks_rdd.filter(lambda x: x.pl_title != None and x.pl_from != None).map(lambda x: (x.pl_title, x.pl_from))

In [ ]:
contents_by_title = contents_rdd.filter(lambda x: x.title != None and x.id != None).map(lambda x: (x.title,x.id))

Armo el grafo 

In [ ]:
graph = pagelinks_by_title.join(contents_by_title).map(lambda x: (x[1][0], int(x[1][1]))).cache()

In [ ]:
graph.take(5)

[(4931539, 7722),
 (4409976, 7722),
 (2570696, 7722),
 (6599661, 7722),
 (7840823, 7722)]

In [ ]:
graph.count()

673026

Funcion que dado el grafo y un numero de id, devuelve una lista de ids a donde referencia

In [ ]:
def ids_a_un_grado(graph, id):

  return graph.filter(lambda x: x[0] == id).map(lambda x: x[1]).collect()

Quiero saber a cuantos id referencian algunas paginas para probar la funcion con un par de esos ids

In [ ]:
graph.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y).take(15)

[(5023935, 1),
 (9417576, 1),
 (8198624, 2),
 (33782, 1),
 (1419348, 2),
 (24745, 1),
 (5348245, 1),
 (7879830, 1),
 (6120660, 1),
 (309617, 2),
 (705124, 1),
 (1613990, 2),
 (5100256, 3),
 (1048628, 2),
 (4416608, 1)]

Pruebo con la funcion

In [ ]:
ids_a_un_grado(graph, 5100256)

[7722, 372555, 372555]

In [ ]:
ids_a_un_grado(graph, 1048628)

[7722, 8636010]